# Organizations GTFS Status Report

In [1]:
from calitp.tables import tbl
from siuba import *
import pandas as pd

import utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [24]:
import altair as alt
from dla_utils import _dla_utils
from shared_utils import styleguide


In [4]:
from calitp import query_sql, magics

In [5]:
pd.set_option("display.max_columns", 100)

In [6]:
pct_compliant = query_sql(
"""

WITH

complete_status AS (
    select
        caltrans_district,
        count(*) AS `count`
    from  cal-itp-data-infra-staging.staging_views.airtable_orgs_filtered
    where (complete_static_gtfs_coverage = 1)
    group by caltrans_district
),

all_status AS (
    select
        caltrans_district,
        count(*) AS `count`
    from  cal-itp-data-infra-staging.staging_views.airtable_orgs_filtered   
    group by caltrans_district
)

SELECT
    caltrans_district,
    all_status.count as `all_org`,
    complete_status.count as `gtfs_compliant`,
    100.0 * (complete_status.count / all_status.count) AS `percent`
FROM all_status 
join complete_status 
    using (caltrans_district)
    
""")

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [7]:
pct_compliant

,caltrans_district,all_org,gtfs_compliant,percent
0,07 - Los Angeles,55,47,85.45
1,06 - Fresno,14,12,85.71
2,03 - Marysville,18,15,83.33
3,11 - San Diego,6,4,66.67
4,04 - Oakland,34,24,70.59
5,10 - Stockton,14,13,92.86
6,02 - Redding,8,7,87.50
7,05 - San Luis Obispo,13,11,84.62
8,12 - Irvine,6,6,100.00
9,08 - San Bernardino,11,11,100.00


In [39]:
df = query_sql(
"""
WITH new_table AS (
SELECT 
    *,
        CASE 
        WHEN (fares_v2_status LIKE ('%Unknown%') 
              or fares_v2_status is null 
              or fares_v2_status LIKE ('%Blocked%') 
              or fares_v2_status LIKE ('%to Publish%') 
              or fares_v2_status LIKE ('%Need%')) then 0 
        WHEN (fares_v2_status LIKE ('%published%')) then 1
    end AS `fares_v2_status_complete`,
        CASE 
        WHEN (fares_v2_status LIKE ('%Unknown%') or fares_v2_status is null) then 'Unknown'
        WHEN (fares_v2_status LIKE ('%Blocked%') or fares_v2_status LIKE ('%Need%')) then 'Needed'
        WHEN (fares_v2_status LIKE ('%to Publish%') or fares_v2_status LIKE ('%in progress%')) then 'In works'
        WHEN (fares_v2_status LIKE ('%published%')) then 'Published'
    end AS `fares_v2_status2`,
FROM cal-itp-data-infra-staging.staging_views.airtable_orgs_filtered
)

SELECT 
    *,
FROM new_table
LIMIT 10
""")

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [40]:
df.head()

,organization_id,name,mobility_services_managed,parent_organization,organization_type,headquarters_place,caltrans_district,planning_authority,tracking_category,reporting_category,assist_category,headquarters_state_country,record_creation_time,service_type__from_mobility_services_managed_,currently_operating__from_mobility_services_managed_,funding_sources_for_managed_transportation,num_fixed_route_services_w_static_gtfs,at_least_one_gtfs_feed_for_any_service,service_availability_category__from_mobility_services_managed_,num_fixed_route_or_deviated_fixed_route_services,num_fixed_route_or_deviated_fixed_route_service_w__static_gtfs,num_services_with_missing_static_feed_for_fixed_route_or_deviated_fixed_route,num_fixed_route_services,complete_static_gtfs_coverage,gtfs_static_status,num_services_w_complete_rt_status,complete_rt_coverage,at_least_one_complete_rt_set,gtfs_realtime_status,missing_static,services_needing_alerts,services_needing_tripupdates_or_vehiclepositions,service_availability,fixed_route_service_operator_type,schedule_datasets,itp_id,gtfs_datasets_produced,mobility_services_operated,ntp_id,mpo_rtpa,gtfs_datasets_referenced,gtfs_dataset__from_mobility_services_managed_,total_voms__ntd_,service_area_population__ntd_,service_area_sq_miles__ntd_,currently_operating__from_mobility_services_operated_,service_type__from_mobility_services_operated_,gtfs_schedule_status,flex_status,immediate_gtfs_goals,fares_v2_status,details,website,funding_programs,county_geography,fare_systems,drmt_organization_name,drmt_reported_5310_vehicles,opm_id_drmt,alias,service___component,brand,roles,county_geography_3,dotid,administrating_organization,service_area_type__from_services_,avl_rti_vendor__from_service_area_type__from_services__,currently_operating__from_service_area_type__from_services__,services,eligibility_programs,fares_v2_status_complete,fares_v2_status2
0,recC5MAOasVIbzpAV,City of Artesia,"['recwVQU6GFZkfDnSx', 'rectIZqFrrnttOXXX']",None,City/Town,Artesia,07 - Los Angeles,['recuOdDxg9NGNH79X'],Active,Core,White Glove,CA,2021-07-17T00:45:50.000Z,"['ADA paratransit', 'fixed-route']","[True, True]","['public', 'public']",0,0,"['Public\n', 'Public\n']",1,0,1,1,0,Static Incomplete,0,0,0,RT Incomplete,['Artesia Transit'],"['Artesia Express Dial-A-Ride', 'Artesia Trans...","['Artesia Express Dial-A-Ride', 'Artesia Trans...","['Public\n', 'Public\n']",['provider-operated'],0,20.00,None,"['recwVQU6GFZkfDnSx', 'rectIZqFrrnttOXXX']",['reclLTNKOF5lIBJbH'],['recuOdDxg9NGNH79X'],None,None,[5],[12150996],[1736],"[True, True]","['ADA paratransit', 'fixed-route']","['needed', 'needed']",['Needed - Public service'],['0 - Does not have GTFS Schedule'],None,None,None,None,None,['recZ0bJA1WDvrt1yL'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,Unknown
1,recV6WAkk4yeeVZLh,City of Maywood,"['recnLFaMjjcTXyAmN', 'recO2LpNVdWZV6Emn']",None,City/Town,Maywood,07 - Los Angeles,['recuOdDxg9NGNH79X'],Active,Core,White Glove,CA,2021-07-17T00:45:50.000Z,"['ADA paratransit', 'fixed-route']","[True, True]","['public', 'public']",0,0,"['Conditional', 'Public\n']",1,0,1,1,0,Static Incomplete,0,0,0,RT Incomplete,['Maywood Express Shuttle'],"['Maywood Dial-A-Ride', 'Maywood Express Shutt...","['Maywood Dial-A-Ride', 'Maywood Express Shutt...","['Conditional', 'Public\n']",['provider-operated'],0,196.00,None,"['recnLFaMjjcTXyAmN', 'recO2LpNVdWZV6Emn']",['recpjozvZELW1Neeh'],['recuOdDxg9NGNH79X'],None,None,[5],[12150996],[1736],"[True, True]","['ADA paratransit', 'fixed-route']","['needed', 'needed']",['Needed - Special service'],['0 - Does not have GTFS Schedule'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,Unknown
2,recXYM27Lts8XF98x,City of Cerritos,"['recQ975tFNFBKgcL3', 'recAfsm7PMFRhTT43']",None,City/Town,Cerritos,07 - Los Angeles,['recuOdDxg9NGNH79X'],Active,Core,White Glove,CA,2021-07-17T00:45:50.000Z,"['fixed-route', 'ADA paratransit']","[True, True]","['public', 

In [41]:
pct_compliant['not_compliant'] = pct_compliant.all_org - pct_compliant.gtfs_compliant

In [42]:
pct_compliant

,caltrans_district,all_org,gtfs_compliant,percent,not_compliant
0,07 - Los Angeles,55,47,85.45,8
1,06 - Fresno,14,12,85.71,2
2,03 - Marysville,18,15,83.33,3
3,11 - San Diego,6,4,66.67,2
4,04 - Oakland,34,24,70.59,10
5,10 - Stockton,14,13,92.86,1
6,02 - Redding,8,7,87.50,1
7,05 - San Luis Obispo,13,11,84.62,2
8,12 - Irvine,6,6,100.00,0
9,08 - San Bernardino,11,11,100.00,0


In [59]:
(_dla_utils.bar_chart_nosubset(pct_compliant,
                               'caltrans_district',
                               'all_org',
                               'caltrans_district',
                               chart_title='Number of Organizations by District'))

alt.Chart(...)

In [60]:
(_dla_utils.bar_chart_nosubset(pct_compliant,
                               'caltrans_district',
                               'gtfs_compliant',
                               'caltrans_district',
                               chart_title='Number of GTFS Static Compliant Organizations by District'))

alt.Chart(...)

In [45]:
(_dla_utils.bar_chart_nosubset(pct_compliant,
                               'caltrans_district',
                               'percent',
                               'caltrans_district',
                               chart_title='Percent of GTFS Static Compliant Organizations by District'))

alt.Chart(...)

In [46]:
df.sample()

,organization_id,name,mobility_services_managed,parent_organization,organization_type,headquarters_place,caltrans_district,planning_authority,tracking_category,reporting_category,assist_category,headquarters_state_country,record_creation_time,service_type__from_mobility_services_managed_,currently_operating__from_mobility_services_managed_,funding_sources_for_managed_transportation,num_fixed_route_services_w_static_gtfs,at_least_one_gtfs_feed_for_any_service,service_availability_category__from_mobility_services_managed_,num_fixed_route_or_deviated_fixed_route_services,num_fixed_route_or_deviated_fixed_route_service_w__static_gtfs,num_services_with_missing_static_feed_for_fixed_route_or_deviated_fixed_route,num_fixed_route_services,complete_static_gtfs_coverage,gtfs_static_status,num_services_w_complete_rt_status,complete_rt_coverage,at_least_one_complete_rt_set,gtfs_realtime_status,missing_static,services_needing_alerts,services_needing_tripupdates_or_vehiclepositions,service_availability,fixed_route_service_operator_type,schedule_datasets,itp_id,gtfs_datasets_produced,mobility_services_operated,ntp_id,mpo_rtpa,gtfs_datasets_referenced,gtfs_dataset__from_mobility_services_managed_,total_voms__ntd_,service_area_population__ntd_,service_area_sq_miles__ntd_,currently_operating__from_mobility_services_operated_,service_type__from_mobility_services_operated_,gtfs_schedule_status,flex_status,immediate_gtfs_goals,fares_v2_status,details,website,funding_programs,county_geography,fare_systems,drmt_organization_name,drmt_reported_5310_vehicles,opm_id_drmt,alias,service___component,brand,roles,county_geography_3,dotid,administrating_organization,service_area_type__from_services_,avl_rti_vendor__from_service_area_type__from_services__,currently_operating__from_service_area_type__from_services__,services,eligibility_programs,fares_v2_status_complete,fares_v2_status2
5,rec6suFcJnXKb8ARd,City of Beverly Hills,"['reckhgOj3efNz3vi1', 'recr38Kxa6Utl2QCW', 're...",None,City/Town,Beverly Hills,07 - Los Angeles,['recuOdDxg9NGNH79X'],Active,Core,White Glove,CA,2021-07-17T00:45:50.000Z,"['fixed-route', 'ADA paratransit', 'reservatio...","[True, True, True]","['public', 'public', 'public']",0,0,"['Public\n', 'Conditional', 'Conditional']",1,0,1,1,0,Static Incomplete,0,0,0,RT Incomplete,['Beverly Hills Trolley'],"['Beverly Hills Trolley', 'Beverly Hills Dial-...","['Beverly Hills Trolley', 'Beverly Hills Dial-...","['Public\n', 'Conditional', 'Conditional']",['provider-operated'],0,39.00,None,"['reckhgOj3efNz3vi1', 'recr38Kxa6Utl2QCW']",['reciYQ98CHbGKoB7D'],['recuOdDxg9NGNH79X'],None,None,[4],[12150996],[1736],"[True, True]","['fixed-route', 'ADA paratransit', 'reservatio...","['needed', 'needed', 'needed']","['Needed - Special service', 'Needed - Special...",['0 - Does not have GTFS Schedule'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,Unknown


In [53]:
fares_status2 =query_sql(
"""
WITH new_table AS (
SELECT 
    name,
    caltrans_district,
    fares_v2_status,
        CASE 
        WHEN (fares_v2_status LIKE ('%Unknown%') or fares_v2_status is null) then 'Unknown'
        WHEN (fares_v2_status LIKE ('%Blocked%') or fares_v2_status LIKE ('%Need%')) then 'Needed'
        WHEN (fares_v2_status LIKE ('% to Publish%') or fares_v2_status LIKE ('%in progress%')) then 'In works'
        WHEN (fares_v2_status LIKE ('%published%')) then 'Published'
    end AS `fares_v2_status2`
FROM cal-itp-data-infra-staging.staging_views.airtable_orgs_filtered
)

SELECT 
    caltrans_district,
    fares_v2_status2,
    count(fares_v2_status2) AS `count`
FROM new_table
GROUP BY caltrans_district, fares_v2_status2
ORDER BY caltrans_district
""")

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [ ]:
fares_status2.fares_v2_status2.value_counts()

Unknown      12
Published    11
Needed        7
In works      5
Name: fares_v2_status2, dtype: int64

In [63]:
(_dla_utils.bar_chart_nosubset(fares_status2,
                               'caltrans_district',
                               'sum(count)',
                               'fares_v2_status2',
                               chart_title='Fares v2 Status by District'))

alt.Chart(...)